# Generators and Infinite Sequences

It is difficult to work with infinite streams of data, like sequences and series, using standard Python tools.

For instance, the usual way to study terms in the Fibonacci sequence is to write a function that returns all Fibonnaccis up to some upper bound $M$, then get a sufficiently large chunk.

In [32]:
import numpy as np

def fibs_up_to_M(M):
    """
    Returns a list of Fibonacci numbers less than or equal to M
    """
    x, y = 1, 1
    
    fiblist = [1,1]
    
    while fiblist[-1] <= M:
        x, y = y, x+y
        fiblist.append(y)
        
    return fiblist

fibs_up_to_M(100)

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144]

This is a sloppy solution if one wants to study the entire sequence $(a_n)$ of Fibonaccis.

Every time we want to create a larger list up to a differnet maximum value $M'$, we must call this function again and repeat all of the computational work of computing the lower values.

Furthermore, if $M$ is very large, it can be very inefficient to hold the entire list $(a_n)_{n=1}^{M}$ in memory, especially if we're interested in a problem that only needs to look at a single Fibonacci at a time.

A much more efficient way to represent sequences in Python is via _generators_.

In [2]:
def fib_gen():
    """
    Returns a generator that yields Fibonacci numbers
    """
    x, y = 1, 1

    while True:
        yield y
        x, y = y, x+y

Notice the new keyword `yield`. Whenever a function block contains `yield`, Python automatically makes it a generator rather than an ordinary function.

When Python sees a `yield` statement inside a generator, it "freezes the state" of all variables inside the generator. Then, the next time you call the generator, it picks up where you left off -- all variables take the values they had the last time it was called.

In [22]:
my_gen = fib_gen()

next(my_gen)

1

The `next` keyword, predictably, grabs the next element from the generator. Now if we call it again, it continues where it left off.

In [23]:
next(my_gen)

2

In [24]:
for i in range(10):
    print(next(my_gen))

3
5
8
13
21
34
55
89
144
233


Using the generator and `next`, we can work with the values one at a time rather than putting them in a list.

This can be convenient for operations which only need one value at a time, like getting partial sums of a sequence.

In [33]:
def generate_partial_sums(an):
    """
    Takes in a sequence a_n and generates a stream of its partial sums
    """
    sk = 0
    
    while True:
        sk += next(an)
        yield sk

In [ ]:
def geometric_series(r):
    """
    Returns terms in the geometric series r^n
    """
    n = 0
    
    while True:
        yield np.power(r, n)
        n = n+1

In [ ]:
geometric_sum = generate_partial_sums(geometric_series(0.5))

for i in range(10):
    print(next(geometric_sum))

The partial sums are converging to $\sum_{n=1}^{\infty} \left( \frac{1}{2} \right)^n = \frac{1]{1 - \frac{1}{2}} = 2$, as expected.

# Cesaro Summation

A sequence $(a_n)$ is called _Cesaro summable_, with _Cesaro sum_ $\ell$, if

$$\lim_{n \to \infty} \frac{s_1 + \cdots + s_n}{n} = \ell ,$$

where $s_k = a_1 + \cdots + s_k$.

That is: instead of asking for the usual partial sums $s_k$ to converge, Cesaro summation only asks for the _average_ of the partial sums to converge.

***(Part a)*** Every summable series is Cesaro summable, but not every Cesaro summable series is summable.

Construct an example of a series which diverges (in the usual sense) but is still Cesaro summable. Type your answer below.

(Hint: choose an alternating series whose terms don't go to zero in absolute value.)

**Solution:** Consider the sequence 
\begin{align} a_n = (-1)^n\end{align}
Then, suppose by way of contradiction that this series is convergent; then it must be true that $$\lim_{n \to \infty} a_n = 0.$$ Clearly this is false, since $$a_n = \pm 1.$$ However, the partial sums alternate between $-1$ and $0$ every point, so the limit in question equals 
\begin{align} \lim_{n \to \infty} \frac{s_1 + \cdots + s_n}{n} = \lim_{n \to \infty} \frac{\frac{-1}{2}n}{n} = \frac{-1}{2}. \end{align}
Thus, this sequence is both divergent and Cesaro summable (since the limit exists).

***(Part b)*** Write a generator `cesaro_sums` which takes as input a generator `an` that yields terms in some sequence $a_n$, and yields the Cesaro partial sums.

That is, your function should be able to accept the terms $a_n$ one-at-a-time, and return the average of the first $k$ partial sums one-at-a-time.

_Hint_: you may think that you need to store _all_ of the partial sums to compute their average. This is not true. Using some algebra, you can quickly see that it's easy to compute the new average using only the old overage and the new data point. If $\mu_{n-1}$ is the average of the first $n-1$ partial sums, i.e.

$$\mu_{n-1} = \frac{s_1 + \cdots + s_{n-1}}{n-1} , $$

then the average of the first $n$ partial sums is

$$\mu_n = \frac{s_1 + \cdots + s_{n-1}}{n} = \frac{(n-1) \mu_{n-1} + s_n}{n}$$

and therefore

$$ \mu_n = \mu_{n-1} + \frac{s_n - \mu_{n-1}}{n} . $$

This technique of computing the average in a one-at-a-time fashion, rather than keeping all the data points, is called an [online algorithm](https://en.wikipedia.org/wiki/Online_algorithm).

In [50]:
def generate_cesaro_sums(an):
    """
    Takes in a sequence a_n and generates a stream of the averages of its partial sums
    """
    sk = 0
    muk = 0
    iteration = 0
    
    partial_sum_generator = generate_partial_sums(an)
    while True:
        iteration += 1
        yield muk
        sk = next(partial_sum_generator)
        muk = muk + (sk - muk)/iteration

In [53]:
def example_sequence():
    x = -1
    n = 1
    while True:
        yield x**n
        n += 1

In [55]:
test_generator = generate_cesaro_sums(example_sequence())
for x in range(5):
    print(next(test_generator))

0
-1.0
-0.5
-0.6666666666666666
-0.5
